In [11]:
import numpy as np
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
def Load_Data():
    
    data = pd.read_csv("dataset1.csv")
    x = data.iloc[:, :-1]
    x = x.as_matrix()
    y = data.iloc[:, -1]
    y = y.as_matrix()
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    return X_train, Y_train, X_test, Y_test

In [12]:
class L_Layer_Neural_Network:

    def __init__(self,layers):
        self.layers = layers
        self.L = len(self.layers)
        self.parameters = {}
        self.caches = []

    def Sigmoid(self,Z):
        cache = Z
        A = 1/(1+np.exp(-Z))
        return A,cache

    def Relu(self,Z):
        cache = Z
        A = np.maximum(0,Z)
        return A,cache

    def Sigmoid_Backward(self,dA, cache):
        Z = cache
        s = 1/(1+np.exp(-Z))
        dZ = dA*s*(1-s)
        return dZ

    def Relu_Backward(self,dA, cache):
        Z = cache
        dZ = np.array(dA,copy=True)
        dZ[Z <= 0] = 0
        return dZ

    def Initialize_Parameters(self):
        for l in range(1,self.L):
            self.parameters['W'+str(l)] = np.random.randn(self.layers[l],self.layers[l-1])*0.01
            self.parameters['b'+str(l)] = np.zeros((self.layers[l],1))

            assert(self.parameters['W'+str(l)].shape == (self.layers[l],self.layers[l-1])),"Parameters W dimension do not match"
            assert(self.parameters['b'+str(l)].shape == (self.layers[l],1)),"Parameters b dimension do not match"

    def Linear_Forward(self,A,W,b):
        Z = np.dot(W,A)+b
        assert(Z.shape == (W.shape[0], A.shape[1]))
        cache = (A,W,b)
        return Z,cache

    def Linear_Activation_Forward(self,A_prev,W,b,activation):
        if activation == 'Sigmoid':
            Z, linear_cache = self.Linear_Forward(A_prev,W,b)
            A, activation_cache = self.Sigmoid(Z)
        elif activation == 'Relu':
            Z, linear_cache = self.Linear_Forward(A_prev,W,b)
            A, activation_cache = self.Relu(Z)
        cache = (linear_cache, activation_cache)
        return A, cache

    def Forward_Propagation(self,X):
        self.caches = []
        A = X
        L = self.L
        m = X.shape[1]
        for l in range(1,L-1):
            A_prev = A
            W = self.parameters['W'+str(l)]
            b = self.parameters['b'+str(l)]
            A, cache = self.Linear_Activation_Forward(A_prev,W,b,'Relu')
            self.caches.append(cache)
            
        W = self.parameters['W'+str(L-1)]
        b = self.parameters['b'+str(L-1)]
        AL, cache = self.Linear_Activation_Forward(A,W,b,'Sigmoid')
        self.caches.append(cache)
        return AL

    def Compute_Cost(self,AL):
        cost = -1*(np.sum(np.multiply(self.Y,np.log(AL)) + np.multiply(1-self.Y,np.log(1-AL))))/self.m
        cost = np.squeeze(cost)
        return cost

    def Linear_Backward(self,dZ,cache):
        A_prev, W, b = cache
        dW = (np.dot(dZ,A_prev.T))/self.m
        db = np.sum(dZ,axis=1,keepdims=True)/self.m
        dA_prev = np.dot(W.T,dZ)
        return dA_prev, dW, db

    def Linear_Activation_Backward(self,dA, cache, activation):
        linear_cache, activation_cache = cache
        if activation == 'Relu':
            dZ = self.Relu_Backward(dA, activation_cache)
            dA_prev, dW, db = self.Linear_Backward(dZ, linear_cache)
        if activation == 'Sigmoid':
            dZ = self.Sigmoid_Backward(dA, activation_cache)
            dA_prev, dW, db = self.Linear_Backward(dZ, linear_cache)
        return dA_prev, dW, db

    def Backward_Propagation(self,AL):
        self.grads = {}
        dAL = -(np.divide(self.Y,AL) - np.divide(1-self.Y,1-AL))
        current_cache = self.caches[self.L-2]
        self.grads["dA"+str(self.L-1)], self.grads["dW"+str(self.L-1)], self.grads["db"+str(self.L-1)] = self.Linear_Activation_Backward(dAL,current_cache,"Sigmoid")
        for l in reversed(range(self.L-2)):
            current_cache = self.caches[l]
            current_dA = self.grads['dA'+str(l+2)]
            # print(len(current_cache))
            dA_prev_temp, dW_temp, db_temp = self.Linear_Activation_Backward(current_dA,current_cache,"Relu")
            self.grads['dA'+str(l+1)] = dA_prev_temp
            self.grads['dW'+str(l+1)] = dW_temp
            self.grads['db'+str(l+1)] = db_temp

    def Update_Parameters(self):
        for l in range(1,self.L):
            self.parameters["W"+str(l)] = self.parameters["W"+str(l)] - self.learning_rate*self.grads["dW"+str(l)]
            self.parameters["b"+str(l)] = self.parameters["b"+str(l)] - self.learning_rate*self.grads["db"+str(l)]

    def Train(self,X,Y,learning_rate=0.075,num_iterations=3000,print_cost=False):
        self.m = X.shape[0]
        self.X = X.T
        self.Y = Y
        self.learning_rate = learning_rate
        self.Initialize_Parameters()
        self.costs = []
        for i in range(num_iterations):
            AL = self.Forward_Propagation(self.X)
            cost = self.Compute_Cost(AL)
            self.Backward_Propagation(AL)
            self.Update_Parameters()
            if print_cost and i%100 == 0:
                print("Cost after iteration "+str(i)+": "+str(cost))
                self.costs.append(cost)
                # print(self.parameters)
        return self.parameters
    
    def Performance(self, X, Y):
        X = np.array(X).T
        predictions = self.Forward_Propagation(X)
        predictions = np.round(predictions)
        print ('Accuracy: %d' % float((np.dot(Y, predictions.T) + np.dot(1 - Y, 1 - predictions.T)) / float(Y.size) * 100) + '%')
        return

In [14]:
model = L_Layer_Neural_Network([24,64,32,1])
X_train, Y_train, X_test, Y_test = Load_Data()
train_accuracy = model.Train(X_train, Y_train,0.0001,10000,True)
test_accuracy = model.Performance(X_test, Y_test )


c:\users\user\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
c:\users\user\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


Cost after iteration 0: 0.6763662016138758
Cost after iteration 100: 0.622425323349023
Cost after iteration 200: 0.621380154520385
Cost after iteration 300: 0.6212450731694789
Cost after iteration 400: 0.6211155000098603
Cost after iteration 500: 0.6209854041745977
Cost after iteration 600: 0.6208546509678115
Cost after iteration 700: 0.6207231498365972
Cost after iteration 800: 0.6205908014201477
Cost after iteration 900: 0.6204574868260516
Cost after iteration 1000: 0.620323102308578
Cost after iteration 1100: 0.6201874993552342
Cost after iteration 1200: 0.6200505667209237
Cost after iteration 1300: 0.6199121982189983
Cost after iteration 1400: 0.619772138274305
Cost after iteration 1500: 0.6196304373124104
Cost after iteration 1600: 0.6194868884088895
Cost after iteration 1700: 0.6193413399125105
Cost after iteration 1800: 0.619193635624633
Cost after iteration 1900: 0.6190436158621386
Cost after iteration 2000: 0.6188911061043872
Cost after iteration 2100: 0.6187359170483613
Cost 